In [1]:
from keras.layers import Input, Dense
from keras.models import Model

# this is the size of our encoded representations
encoding_dim = 32  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# this is our input placeholder
input_img = Input(shape=(784,))
# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation='relu')(input_img)
# "decoded" is the lossy reconstruction of the input
decoded = Dense(784, activation='sigmoid')(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)

Using TensorFlow backend.


In [2]:
# this model maps an input to its encoded representation
encoder = Model(input_img, encoded)

In [3]:
# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))

In [4]:
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [5]:
from keras.datasets import mnist
import numpy as np
(x_train, _), (x_test, _) = mnist.load_data()

In [7]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print (x_train.shape)
print (x_test.shape)

(60000, 784)
(10000, 784)


In [8]:
autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 4s 72us/step - loss: 0.3615 - val_loss: 0.2712
Epoch 2/50
60000/60000 [==============================] - 4s 63us/step - loss: 0.2643 - val_loss: 0.2539
Epoch 3/50
60000/60000 [==============================] - 4s 69us/step - loss: 0.2443 - val_loss: 0.2324
Epoch 4/50
60000/60000 [==============================] - 4s 71us/step - loss: 0.2247 - val_loss: 0.2142
Epoch 5/50
60000/60000 [==============================] - 4s 61us/step - loss: 0.2087 - val_loss: 0.2004ETA: 0s - lo
Epoch 6/50
60000/60000 [==============================] - 3s 54us/step - loss: 0.1967 - val_loss: 0.1903
Epoch 7/50
60000/60000 [==============================] - 3s 53us/step - loss: 0.1877 - val_loss: 0.1823
Epoch 8/50
60000/60000 [==============================] - 3s 53us/step - loss: 0.1805 - val_loss: 0.1757
Epoch 9/50
60000/60000 [==============================] - 3s 53us/step - loss: 0.1746 - val_loss: 

In [9]:
# encode and decode some digits
# note that we take them from the *test* set
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)

In [10]:
# use Matplotlib (don't ask)
import matplotlib.pyplot as plt

n = 10  # how many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

<Figure size 2000x400 with 20 Axes>

In [13]:
from keras import regularizers

encoding_dim = 32

input_img = Input(shape=(784,))
# add a Dense layer with a L1 activity regularizer
encoded = Dense(encoding_dim, activation='relu',
                activity_regularizer=regularizers.l1(10e-5))(input_img)
decoded = Dense(784, activation='sigmoid')(encoded)

autoencoder = Model(input_img, decoded)

In [14]:
input_img = Input(shape=(784,))
encoded = Dense(128, activation='relu')(input_img)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(32, activation='relu')(encoded)

decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(784, activation='sigmoid')(decoded)

In [15]:
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

autoencoder.fit(x_train, x_train,
                epochs=100,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/100
60000/60000 [==============================] - 6s 98us/step - loss: 0.3486 - val_loss: 0.2639
Epoch 2/100
60000/60000 [==============================] - 5s 86us/step - loss: 0.2592 - val_loss: 0.2549
Epoch 3/100
60000/60000 [==============================] - 5s 80us/step - loss: 0.2487 - val_loss: 0.2416
Epoch 4/100
60000/60000 [==============================] - 5s 83us/step - loss: 0.2349 - val_loss: 0.2269
Epoch 5/100
60000/60000 [==============================] - 5s 87us/step - loss: 0.2215 - val_loss: 0.2145
Epoch 6/100
60000/60000 [==============================] - 5s 85us/step - loss: 0.2106 - val_loss: 0.2055
Epoch 7/100
60000/60000 [==============================] - 5s 81us/step - loss: 0.1971 - val_loss: 0.1881
Epoch 8/100
60000/60000 [==============================] - 5s 89us/step - loss: 0.1850 - val_loss: 0.1811
Epoch 9/100
60000/60000 [==============================] - 6s 93us/step - loss: 0.1789 - val_loss: 0.1

60000/60000 [==============================] - 5s 83us/step - loss: 0.1051 - val_loss: 0.1033
Epoch 78/100
60000/60000 [==============================] - 5s 84us/step - loss: 0.1050 - val_loss: 0.1048
Epoch 79/100
60000/60000 [==============================] - 5s 83us/step - loss: 0.1047 - val_loss: 0.1042
Epoch 80/100
60000/60000 [==============================] - 5s 84us/step - loss: 0.1047 - val_loss: 0.1034
Epoch 81/100
60000/60000 [==============================] - 6s 92us/step - loss: 0.1044 - val_loss: 0.1063
Epoch 82/100
60000/60000 [==============================] - 6s 93us/step - loss: 0.1041 - val_loss: 0.1039
Epoch 83/100
60000/60000 [==============================] - 5s 78us/step - loss: 0.1040 - val_loss: 0.1041
Epoch 84/100
60000/60000 [==============================] - 5s 79us/step - loss: 0.1037 - val_loss: 0.1028
Epoch 85/100
60000/60000 [==============================] - 5s 76us/step - loss: 0.1036 - val_loss: 0.1022
Epoch 86/100
60000/60000 [========================